In [81]:
import os
import glob
import numpy
import pickle
import tensorflow as tf

from music21 import converter, instrument, note, chord
from keras.utils import np_utils

In [82]:
notes = []

In [83]:
for file in glob.glob("dataset/midi/*.mid"):
    midi = converter.parse(file)

    notes_to_parse = None

    try:
        s2 = instrument.partitionByInstrument(midi)
        notes_to_parse = s2.parts[0].recurse() 
    except:
        notes_to_parse = midi.flat.notes

    for element in notes_to_parse:
        if isinstance(element, note.Note):
            notes.append(str(element.pitch))
        elif isinstance(element, chord.Chord):
            notes.append('.'.join(str(n) for n in element.normalOrder))

with open('dataset/notes/parsed.data', 'wb') as filepath:
    pickle.dump(notes, filepath)

In [84]:
sequence_length = 100
n_vocab = len(set(notes))

pitchnames = sorted(set(item for item in notes))

note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

network_input = []
network_output = []

for i in range(0, len(notes) - sequence_length, 1):
    sequence_in = notes[i:i + sequence_length]
    sequence_out = notes[i + sequence_length]
    network_input.append([note_to_int[char] for char in sequence_in])
    network_output.append(note_to_int[sequence_out])

n_patterns = len(network_input)

network_input = numpy.reshape(network_input, (n_patterns, sequence_length, 1))

network_input = network_input / float(n_vocab)

network_output = np_utils.to_categorical(network_output)

In [85]:
model = tf.keras.models.Sequential([
    tf.keras.layers.LSTM(
        512, 
        input_shape=(network_input.shape[1], network_input.shape[2]), 
        recurrent_dropout=0.3, 
        return_sequences=True
    ),
    tf.keras.layers.LSTM(512, return_sequences=True, recurrent_dropout=0.3),
    tf.keras.layers.LSTM(512),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(256),
    tf.keras.layers.Activation(tf.keras.activations.relu),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(n_vocab),
    tf.keras.layers.Activation(tf.keras.activations.softmax)
])

    
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

In [86]:
checkpoint_path = "models/weights-{epoch:02d}-{loss:.4f}.hdf5" 
log_path = "logs/"

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    checkpoint_path, 
    monitor='loss', 
    verbose=0,        
    save_best_only=True,        
    mode='min'
)

tensorboard_callback = tf.keras.callbacks.TensorBoard(log_path, histogram_freq=1)

model.fit(network_input, network_output, epochs=200, batch_size=64, callbacks=[checkpoint_callback, tensorboard_callback])

Epoch 1/200
704/704 [==============================] - 1213s 2s/step - loss: 5.0996
Epoch 2/200
704/704 [==============================] - 1314s 2s/step - loss: 4.6590
Epoch 3/200
704/704 [==============================] - 1319s 2s/step - loss: 4.6171
Epoch 4/200
704/704 [==============================] - 1169s 2s/step - loss: 4.6077
Epoch 5/200
704/704 [==============================] - 1164s 2s/step - loss: 4.5961
Epoch 6/200
704/704 [==============================] - 1161s 2s/step - loss: 4.5791
Epoch 7/200
704/704 [==============================] - 1163s 2s/step - loss: 4.5478
Epoch 8/200
704/704 [==============================] - 1158s 2s/step - loss: 4.5602
Epoch 9/200
704/704 [==============================] - 1157s 2s/step - loss: 4.5188
Epoch 10/200
704/704 [==============================] - 1160s 2s/step - loss: 4.5071
Epoch 11/200
704/704 [==============================] - 1167s 2s/step - loss: 4.4948
Epoch 12/200
704/704 [==============================] - 1157s 2s/step - lo

704/704 [==============================] - 1303s 2s/step - loss: 0.9423
Epoch 98/200
704/704 [==============================] - 1343s 2s/step - loss: 0.9156
Epoch 99/200
704/704 [==============================] - 1207s 2s/step - loss: 0.9117
Epoch 100/200
704/704 [==============================] - 1154s 2s/step - loss: 0.8904
Epoch 101/200
704/704 [==============================] - 1164s 2s/step - loss: 0.8739
Epoch 102/200
704/704 [==============================] - 1160s 2s/step - loss: 0.8626
Epoch 103/200
704/704 [==============================] - 1161s 2s/step - loss: 0.8460
Epoch 104/200
704/704 [==============================] - 1164s 2s/step - loss: 0.8356
Epoch 105/200
704/704 [==============================] - 1162s 2s/step - loss: 0.8328
Epoch 106/200
704/704 [==============================] - 1164s 2s/step - loss: 0.8252
Epoch 107/200
704/704 [==============================] - 1168s 2s/step - loss: 0.8195
Epoch 108/200
704/704 [==============================] - 1164s 2s/step

704/704 [==============================] - 1154s 2s/step - loss: 0.4659
Epoch 193/200
704/704 [==============================] - 1155s 2s/step - loss: 0.4591
Epoch 194/200
704/704 [==============================] - 1155s 2s/step - loss: 0.4464
Epoch 195/200
704/704 [==============================] - 1153s 2s/step - loss: 0.4526
Epoch 196/200
704/704 [==============================] - 1155s 2s/step - loss: 0.4546
Epoch 197/200
704/704 [==============================] - 1155s 2s/step - loss: 0.4570
Epoch 198/200
704/704 [==============================] - 1152s 2s/step - loss: 0.4487
Epoch 199/200
704/704 [==============================] - 1151s 2s/step - loss: 0.4371
Epoch 200/200
704/704 [==============================] - 1156s 2s/step - loss: 0.4464
